# QDK Interop with OpenQASM

The QDK provides interoperability with OpenQASM 3 programs built upon the core QDK compiler infrastructure.

This core enables integration and local resource estimation without relying on external tools. Users are able to estimate resources for their OpenQASM programs locally (see the [resource estimation with OpenQASM sample notebook](../estimation/estimation-openqasm.ipynb)), leveraging the QDK compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from OpenQASM progams leveraging the [QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for classical instructions available in OpenQASM such as for loops, if statements, switch statements, while loops, binary expresssions, and more.

### Simulating OpenQASM programs

In [ ]:
from qsharp.openqasm import run

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

# We'll pass as_bitstring=True to convert bit[n] to a bitstring in the output.
# Otherwise, the output would be a list of Result values.
run(source, as_bitstring=True)

The OpenQASM programs can also be run with noise just as with Q#.

In [ ]:
from qsharp import DepolarizingNoise
from qsharp.openqasm import run
from qsharp_widgets import Histogram

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

Histogram(run(source, noise=DepolarizingNoise(0.01), as_bitstring=True))


### Compiling OpenQASM to Quantum Intermediate Representation (QIR)

We can directly compile OpenQASM to QIR with the `compile` function.

In [ ]:
from qsharp.openqasm import compile

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

compilation = compile(source)

print(compilation)

> For parameterized circuits the `import_openqasm` function must be used to first create a Python callable. A sample parameterized circuit can be found later in this notebook.

### Run OpenQASM 3 Code in interactive session

Import the `qsharp` module.

This initializes a QDK interpreter singleton.

In [ ]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

With the runtime initialized, we can import an OpenQASM program as a Python callable. Here we'll compile the OpenQASM program to a callable name `"bell"`.

In [ ]:
from qsharp.openqasm import import_openqasm

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

import_openqasm(source, name="bell")

We can now import it via the QDK's Python bindings and run it:

In [ ]:
from qsharp.code import bell
bell()

Additionally, since it is defined in the runtime, we can run it directly from a Q# cell:

In [ ]:
%%qsharp
bell()

This also unlocks all of the other `qsharp` package functionality. Like noisy simulation. Here we'll use the `run` function showing how we can call into the program from Python and display a histogram:

In [ ]:
from qsharp import DepolarizingNoise
from qsharp.openqasm import run
from qsharp.code import bell
from qsharp_widgets import Histogram

Histogram(run(bell, shots=1000, noise=DepolarizingNoise(0.01)))

We can draw the progam as a textual circuit rendering passing the Python callable into the circuit function:

In [ ]:
from qsharp.code import bell

qsharp.circuit(bell)

In notebooks, we can do a bit better leveraging the circuit widget:

In [ ]:
from qsharp.code import bell
from qsharp_widgets import Circuit

Circuit(qsharp.circuit(bell))

And finally when getting ready to submit to hardware, we can compile the program to QIR:

In [ ]:
from qsharp.code import bell

print(qsharp.compile(bell))

We can also define input for the compiled OpenQASM code so that we can parameterize input with imported callables:

In [ ]:
from qsharp import init, TargetProfile
from qsharp.openqasm import import_openqasm

source = """
include "stdgates.inc";
input float theta;
bit[2] c;
qubit[2] q;
rx(theta) q[0];
rx(-theta) q[1];
c = measure q;
"""

init(target_profile=TargetProfile.Base)
import_openqasm(source, name="parameterized_program")


In [ ]:
from qsharp.code import parameterized_program
from qsharp.openqasm import compile

print(compile(parameterized_program, 1.57))